## Тема 2.1 Введение в ML. Основные задачи, виды классических моделей, метрики качества моделей.

### 1. Обучение первой модели

_Характеристики набора данных_:

- **Количество экземпляров**: 150 (по 50 в каждом из трех классов)

- **Количество атрибутов**: 4 числовых, прогнозных атрибута и класс

- **Информация об атрибутах**:

    - длина чашелистика (sepal) в см
    - ширина чашелистика (petal) в см
    - длина лепестка в см
    - ширина лепестка в см
    - класс:
        - Ирис-Сетоса (setosa)
        - Ирис-разноцветный (versicolor)
        - Ирис-Вирджиния (virginica)

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree

from matplotlib import pyplot as plt

import pandas as pd

In [ ]:
df = load_iris(as_frame=True).frame

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['target'].value_counts()

In [ ]:
for t in df['target'].unique():
    print(f'==Class {t}==')
    display(df.query(f'target == {t}').describe().loc[['min', 'mean', 'max'], :])

#### Задача 1.1
Многие библиотеки машинного обучения требуют, чтобы признаки были сохранены в отдельных переменных.  
Объявите две переменные: 
- features — запишите в неё признаки;
- target — целевой признак.  

Выведите на экран размеры этих переменных.

In [ ]:
features = df.drop(['target'], axis=1)
target = df['target']

print(features.shape)
print(target.shape)

#### Задача 1.2.
Начнем с обучать модель. Попробуем предсказать результат с помощью дерева. Чтобы запустить обучение, вызовите метод fit() и передайте ему как параметр данные.

In [ ]:
model = tree.DecisionTreeClassifier()
model.fit(features, target)

print(model)

#### Задача 1.3.
Теперь в переменной model полноценная модель.   
Создайте три новых объекта и посмотрите на результаты предсказаний.   
Чтобы предсказать ответы, нужно вызвать метод predict() и передать ему таблицу с признаками новых объектов.  

In [ ]:
# взяли средние значения признаков для каждого из трех классов
new_features = pd.DataFrame(
    [[5.00600, 3.428000, 1.462000, 0.246000],
     [5.936000, 2.770000, 4.260000, 1.326000],
     [6.58800, 2.974000, 5.552000, 2.02600]],
    columns=features.columns)

answers = model.predict(new_features) 
print(answers)

#### Задача 1.4.
Изучите правила, сформированные моделью и оцените их адекватность.  
Правила можно посмотреть, визуализировав дерево с помощью встроенной функции plot_tree.

In [ ]:
plt.figure(figsize=[12, 6])
tree.plot_tree(model, feature_names=features.columns);

**Итог**
- познакомились с библиотекой scikit learn, и научились доставать из нее нужные модели и данные
- научились делить датасет на фичи и целевой признак
- научились обучать модель и получать с помощью нее предсказания
- попробовали интерпретировать правила, сформированные обученной моделью

### 2. Оцениваем качество модели

Продолжаем использовать датасет из предыдущего блока и решать задачу предсказания класса ириса по его параметрам.  
Научимся оценивать качество модели.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

#### Задача 2.1.

Поделите входные данные на обучающую и тестовую части в соотношении 70/30.  
Оцените равномерность деления, сравнив распределение по классам в выборках.

In [ ]:
features_train, features_val, target_train, target_val = train_test_split(features, target, test_size=0.3, random_state=45)

In [ ]:
target_train.value_counts(normalize=True).sort_index()

In [ ]:
target_val.value_counts(normalize=True).sort_index()

#### Задача 2.2  
Обучите модель на тренировочной части датасета.  
Постройте прогноз для тестовой части датасета.  
Посчитайте количество ошибочно расставленных лейблов. Для этого реализуйте функцию, которая:
 - склеит реальные лейблы с прогнозными (zip)
 - в цикле расставит 1 для элементов, у которых лейблы не совпали и 0 для остальных
 - сосчитает сумму единиц, то есть количесто элементов, для которых лейбл выставлен ошибочно

In [ ]:
model = tree.DecisionTreeClassifier()
model.fit(features_train, target_train)
predictions_val = model.predict(features_val)

def count_errors(true_answers, pred_answers):
    all_answers_together = zip(true_answers, pred_answers)
    errors_list = [1 if v[0] != v[1] else 0 for v in all_answers_together]
    return sum(errors_list)

print("Ошибок:", count_errors(target_val, predictions_val))

#### Задача 2.3  
Рассчитайте значение метрики accuracy для обученной модели на тестовых данных.
Для этого реализуйте функцию, которая будет похожа на функцию, рассчитывающую количество ошибок, но вместо количества неправильных ответов, она будет возвращать долю правильных ответов.

In [ ]:
def accuracy(true_answers, pred_answers):
    all_answers_together = zip(true_answers, pred_answers)
    correct_answers_list = [1 if v[0] == v[1] else 0 for v in all_answers_together]
    return sum(correct_answers_list) / len(correct_answers_list)

print("Accuracy:", accuracy(target_val, predictions_val))

#### Задача 2.4
Рассчитайте значение метрики accuracy с помощью библиотечной функции. Сравните результат с результатом, полученным с помощью собственной функции. А также сравните значение метрик для прогнозов на тестовой выборе и на обучающей.

In [ ]:
print("Accuracy test built-in:", accuracy_score(target_val, predictions_val))

predictions_train = model.predict(features_train)
print("Accuracy train built-in:", accuracy_score(target_train, predictions_train))

#### Задача 2.5
Составьте матрицу ошибок и выведите ее.  
Рассчитайте точность и покрытие прогноза класса "2" (virginica).

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(target_val, predictions_val), display_labels=['setosa', 'versicolor', 'virginica']).plot()

In [ ]:
virginica_tp = 13
virginica_true_total = 15
virginica_prediction_total = 13
virginica_precision = virginica_tp / virginica_prediction_total
virginica_recall = virginica_tp / virginica_true_total
print('Virginica precision', virginica_precision)
print('Virginica recall', virginica_recall)

**Итог**
- научились выделять валидационную часть из данных
- научились импортировать библиотечные метрики 
- разобрались в устройстве некоторых метрик, реализовали их
- научились интерпретировать метрики